In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler

In [ ]:
def replace_zeros(df):
    """Заменяет нули на моду, исключая 0 как моду."""
    for col in df.select_dtypes(include=np.number):
        counts = df[col].value_counts()
        mode = counts.drop(0, errors='ignore').nlargest(1).index[0] if len(counts)>1 else df[col].mean()
        df[col] = df[col].replace(0, mode)
    return df

Масштабирование данных.

In [ ]:
data = pd.read_csv('diabetes.csv')

X = data.drop('Outcome', axis=1)
y = data['Outcome']

X = replace_zeros(X)

# Балансировка данных
ros = RandomOverSampler(random_state=294)
X_resampled, y_resampled = ros.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=294)

Построение моделей

Градиентный бустинг

In [ ]:
from sklearn.model_selection import GridSearchCV

# Задаем диапазон значений n_estimators для поиска
param_grid_boosting = {'n_estimators': range(50, 400, 50)} # от 50 до 200 с шагом 50

# Создаем GradientBoostingClassifier
gb_classifier = GradientBoostingClassifier(random_state=42)

# Используем GridSearchCV для подбора n_estimators
grid_boosting = GridSearchCV(gb_classifier, param_grid_boosting, cv=5, scoring='accuracy', n_jobs=-1)
grid_boosting.fit(X_train, y_train)

# Лучшие параметры
print("Лучшее количество базовых алгоритмов:", grid_boosting.best_params_)
print("Лучшая точность:", grid_boosting.best_score_)

# Оценка лучшей модели на тестовом наборе
best_boosting_model = grid_boosting.best_estimator_
y_pred = best_boosting_model.predict(X_test)
print("\nОтчет о классификации на тестовом наборе:")
print(classification_report(y_test, y_pred))

Лучшее количество базовых алгоритмов: {'n_estimators': 250}
Лучшая точность: 0.81875

Отчет о классификации на тестовом наборе:
              precision    recall  f1-score   support

           0       0.86      0.81      0.83       104
           1       0.80      0.85      0.83        96

    accuracy                           0.83       200
   macro avg       0.83      0.83      0.83       200
weighted avg       0.83      0.83      0.83       200



Адаптивный бустинг

In [ ]:
from sklearn.model_selection import GridSearchCV

# Задаем диапазон значений n_estimators для поиска
param_grid_boosting = {'n_estimators': range(2, 100, 10)}

# Создаем GradientBoostingClassifier
gb_classifier = AdaBoostClassifier(random_state=42)

# Используем GridSearchCV для подбора n_estimators
grid_boosting = GridSearchCV(gb_classifier, param_grid_boosting, cv=5, scoring='accuracy', n_jobs=-1)
grid_boosting.fit(X_train, y_train)

# Лучшие параметры
print("Лучшее количество базовых алгоритмов:", grid_boosting.best_params_)
print("Лучшая точность:", grid_boosting.best_score_)

# Оценка лучшей модели на тестовом наборе
best_boosting_model = grid_boosting.best_estimator_
y_pred = best_boosting_model.predict(X_test)
print("\nОтчет о классификации на тестовом наборе:")
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Лучшее количество базовых алгоритмов: {'n_estimators': 52}
Лучшая точность: 0.7887500000000001

Отчет о классификации на тестовом наборе:
              precision    recall  f1-score   support

           0       0.77      0.85      0.81       104
           1       0.81      0.73      0.77        96

    accuracy                           0.79       200
   macro avg       0.79      0.79      0.79       200
weighted avg       0.79      0.79      0.79       200

